In [33]:
import pandas as pd
import numpy as np

In [34]:
glasses = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/glasses.csv')

SELECT *
FROM 
    files.smartwatch2
WHERE 
    source IN ('accelerometer', 'gravity', 'linear_acceleration', 'gyroscope')

Filtering the smartwatch sources and excluding battery. 

In [35]:
smartwatch_sources = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/source_smartwatch_full.csv')

Filter report file unlimited. 

SELECT activity_type, duration, from_1, to_1
FROM files.report2;

In [36]:
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')

In [37]:
# Convert timestamp columns to datetime objects
activity_type_full['from_1'] = pd.to_datetime(activity_type_full['from_1'])
smartwatch_sources['timestamp'] = pd.to_datetime(smartwatch_sources['timestamp'])

# Merge the data frames based on timestamps
merged_data = pd.merge_asof(smartwatch_sources.sort_values('timestamp'), 
                             activity_type_full.sort_values('from_1'),
                             left_on='timestamp',
                             right_on='from_1',
                             direction='backward')

# Optionally, drop redundant columns or rename them as needed
merged_data.drop(columns=['from_1', 'to_1'], inplace=True)
merged_data.rename(columns={'activity_type': 'activity'}, inplace=True)

# Fill NaN values with 0s
merged_data.fillna(0, inplace=True)

# Print or further process the merged data
print(merged_data)

        index               source               timestamp  \
0      164350        accelerometer 2017-06-29 07:46:01.263   
1      164351              gravity 2017-06-29 07:46:01.324   
2      164352  linear_acceleration 2017-06-29 07:46:01.329   
3      164353            gyroscope 2017-06-29 07:46:01.402   
4      164359        accelerometer 2017-06-29 07:46:06.270   
...       ...                  ...                     ...   
27405  150414        accelerometer 2017-07-13 20:05:00.933   
27406  150599            gyroscope 2017-07-13 20:21:31.322   
27407  164176            gyroscope 2017-07-13 20:21:31.322   
27408  150603        accelerometer 2017-07-13 20:21:31.358   
27409  164180        accelerometer 2017-07-13 20:21:31.358   

                                               values_1     activity duration  
0              ['-0.3440857', '0.68330383', '9.967361']            0        0  
1              ['-0.34131828', '0.6573288', '9.778641']            0        0  
2       ['-0.01

In [38]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_data = merged_data[(merged_data['activity'] != 0) & (merged_data['duration'] != 0)]

# Print or further process the filtered data
print(filtered_data)

        index               source               timestamp  \
1122   169367            gyroscope 2017-06-29 09:57:08.396   
1123   169370        accelerometer 2017-06-29 09:57:08.406   
1124   169375              gravity 2017-06-29 09:57:08.589   
1125   169376  linear_acceleration 2017-06-29 09:57:08.591   
1126   169379        accelerometer 2017-06-29 09:57:13.436   
...       ...                  ...                     ...   
27405  150414        accelerometer 2017-07-13 20:05:00.933   
27406  150599            gyroscope 2017-07-13 20:21:31.322   
27407  164176            gyroscope 2017-07-13 20:21:31.322   
27408  150603        accelerometer 2017-07-13 20:21:31.358   
27409  164180        accelerometer 2017-07-13 20:21:31.358   

                                         values_1     activity duration  
1122    ['-0.3515625', '0.68681335', '0.4768982']          Eat    00:20  
1123    ['-10.173126', '-1.4712372', '1.9243927']          Eat    00:20  
1124        ['4.585228', '-8.1015

In [39]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
label_encoder = LabelEncoder()

# Apply label encoding to the "source" column
filtered_data['source_encoded'] = label_encoder.fit_transform(filtered_data['source'])
source_encoder_dict = dict(zip(filtered_data['source'], filtered_data['source_encoded']))

print("\nSource Encoding Dictionary:")
print(source_encoder_dict)



Source Encoding Dictionary:
{'gyroscope': 2, 'accelerometer': 0, 'gravity': 1, 'linear_acceleration': 3}


/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/601277845.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['source_encoded'] = label_encoder.fit_transform(filtered_data['source'])


In [40]:
filtered_filtered_data = filtered_data.drop(columns =['source'])

In [41]:
# Convert 'timestamp' column to datetime format
filtered_filtered_data['timestamp'] = pd.to_datetime(filtered_filtered_data['timestamp'])

# Parse values in 'values_1' column to extract numerical values
filtered_filtered_data['values_1'] = filtered_filtered_data['values_1'].apply(lambda x: [float(val.strip("[]'")) for val in x.split(", ")])

In [42]:
# Convert the 'values_1' column to numeric if possible
filtered_filtered_data['values_1'] = filtered_filtered_data['values_1'].apply(pd.to_numeric, errors='coerce')

# Pivot the data to create columns for each unique source_encoded
pivot_data = pd.pivot_table(filtered_filtered_data, 
                            values='values_1', 
                            index='timestamp', 
                            columns='source_encoded')

# Print the pivoted data
print(pivot_data)

source_encoded                                             0  \
timestamp                                                      
2017-06-29 09:57:08.396                                  NaN   
2017-06-29 09:57:08.406  [-10.173126, -1.4712372, 1.9243927]   
2017-06-29 09:57:08.589                                  NaN   
2017-06-29 09:57:08.591                                  NaN   
2017-06-29 09:57:13.436   [4.726227, -10.251968, 0.49110413]   
...                                                      ...   
2017-07-13 20:03:58.637   [-2.270111, -8.976822, -3.0343475]   
2017-07-13 20:05:00.880                                  NaN   
2017-07-13 20:05:00.933    [-3.1450653, -8.022736, -4.60495]   
2017-07-13 20:21:31.322                                  NaN   
2017-07-13 20:21:31.358   [-0.6756897, 2.7004242, 10.965744]   

source_encoded                                         1  \
timestamp                                                  
2017-06-29 09:57:08.396                        

In [43]:
# Replace 'source_encoded' with 'source'
pivot_data.columns.name = 'source'


In [44]:
# Group 'filtered_filtered_data' by both 'timestamp' and 'activity_encoded' and aggregate 'duration'
duration_grouped = filtered_data.groupby(['timestamp', 'activity'])['duration'].first()

# Merge 'duration_grouped' with 'pivot_data' based on timestamp index
pivot_data = pivot_data.merge(duration_grouped, how='left', on='timestamp')

# Print the updated pivoted data with 'duration' as a column specific to each activity
print(pivot_data)

                                                           0  \
timestamp                                                      
2017-06-29 09:57:08.396                                  NaN   
2017-06-29 09:57:08.406  [-10.173126, -1.4712372, 1.9243927]   
2017-06-29 09:57:08.589                                  NaN   
2017-06-29 09:57:08.591                                  NaN   
2017-06-29 09:57:13.436   [4.726227, -10.251968, 0.49110413]   
...                                                      ...   
2017-07-13 20:03:58.637   [-2.270111, -8.976822, -3.0343475]   
2017-07-13 20:05:00.880                                  NaN   
2017-07-13 20:05:00.933    [-3.1450653, -8.022736, -4.60495]   
2017-07-13 20:21:31.322                                  NaN   
2017-07-13 20:21:31.358   [-0.6756897, 2.7004242, 10.965744]   

                                                       1  \
timestamp                                                  
2017-06-29 09:57:08.396                        

In [45]:
# Replace the encoded column names with their original names
source_names = {0: 'accelerometer', 1: 'gravity', 2: 'gyroscope', 3: 'linear_acceleration'}  # Replace with actual source names
pivot_data.rename(columns=source_names, inplace=True)

# Replace NaN values with 0s
pivot_data.fillna(0, inplace=True)

# Print the updated pivoted data
print(pivot_data)

                                               accelerometer  \
timestamp                                                      
2017-06-29 09:57:08.396                                    0   
2017-06-29 09:57:08.406  [-10.173126, -1.4712372, 1.9243927]   
2017-06-29 09:57:08.589                                    0   
2017-06-29 09:57:08.591                                    0   
2017-06-29 09:57:13.436   [4.726227, -10.251968, 0.49110413]   
...                                                      ...   
2017-07-13 20:03:58.637   [-2.270111, -8.976822, -3.0343475]   
2017-07-13 20:05:00.880                                    0   
2017-07-13 20:05:00.933    [-3.1450653, -8.022736, -4.60495]   
2017-07-13 20:21:31.322                                    0   
2017-07-13 20:21:31.358   [-0.6756897, 2.7004242, 10.965744]   

                                                 gravity  \
timestamp                                                  
2017-06-29 09:57:08.396                        

In [46]:
pivot_data.reset_index(inplace=True)


In [47]:
pivot_data

,timestamp,accelerometer,gravity,gyroscope,linear_acceleration,duration
0,2017-06-29 09:57:08.396,0,0,"[-0.3515625, 0.68681335, 0.4768982]",0,00:20
1,2017-06-29 09:57:08.406,"[-10.173126, -1.4712372, 1.9243927]",0,0,0,00:20
2,2017-06-29 09:57:08.589,0,"[4.585228, -8.101572, 3.083927]",0,0,00:20
3,2017-06-29 09:57:08.591,0,0,0,"[-0.2807727, -0.710495, -0.6929529]",00:20
4,2017-06-29 09:57:13.436,"[4.726227, -10.251968, 0.49110413]",0,0,0,00:20
...,...,...,...,...,...,...
24898,2017-07-13 20:03:58.637,"[-2.270111, -8.976822, -3.0343475]",0,0,0,00:41
24899,2017-07-13 20:05:00.880,0,0,"[0.075683594, -0.002960205, 0.047027588]",0,00:41
24900,2017-07-13 20:05:00.933,"[-3.1450653, -8.022736, -4.60495]",0,0,0,00:41
24901,2017-07-13 20:21:31.322,0,0,"[-0.38728333, 0.05871582, 0.5663452]",0,00:41


In [48]:
# Expand accelerometer column into x, y, z components
accelerometer_components = pivot_data['accelerometer'].apply(pd.Series)
accelerometer_components.columns = ['accel_x', 'accel_y', 'accel_z']

# Expand gravity column into x, y, z components
gravity_components = pivot_data['gravity'].apply(pd.Series)
gravity_components.columns = ['gravity_x', 'gravity_y', 'gravity_z']

# Expand gyroscope column into x, y, z components
gyroscope_components = pivot_data['gyroscope'].apply(pd.Series)
gyroscope_components.columns = ['gyro_x', 'gyro_y', 'gyro_z']

# Expand linear_acceleration column into x, y, z components
linear_acceleration_components = pivot_data['linear_acceleration'].apply(pd.Series)
linear_acceleration_components.columns = ['linear_accel_x', 'linear_accel_y', 'linear_accel_z']

# Concatenate the new components with the original DataFrame
pivot_data = pd.concat([pivot_data, accelerometer_components, gravity_components, gyroscope_components, linear_acceleration_components], axis=1)

# Drop the original columns
pivot_data.drop(columns=['accelerometer', 'gravity', 'gyroscope', 'linear_acceleration'], inplace=True)

# Print the updated DataFrame
print(pivot_data)


                    timestamp duration    accel_x    accel_y    accel_z  \
0     2017-06-29 09:57:08.396    00:20   0.000000        NaN        NaN   
1     2017-06-29 09:57:08.406    00:20 -10.173126  -1.471237   1.924393   
2     2017-06-29 09:57:08.589    00:20   0.000000        NaN        NaN   
3     2017-06-29 09:57:08.591    00:20   0.000000        NaN        NaN   
4     2017-06-29 09:57:13.436    00:20   4.726227 -10.251968   0.491104   
...                       ...      ...        ...        ...        ...   
24898 2017-07-13 20:03:58.637    00:41  -2.270111  -8.976822  -3.034347   
24899 2017-07-13 20:05:00.880    00:41   0.000000        NaN        NaN   
24900 2017-07-13 20:05:00.933    00:41  -3.145065  -8.022736  -4.604950   
24901 2017-07-13 20:21:31.322    00:41   0.000000        NaN        NaN   
24902 2017-07-13 20:21:31.358    00:41  -0.675690   2.700424  10.965744   

       gravity_x  gravity_y  gravity_z    gyro_x    gyro_y    gyro_z  \
0       0.000000        NaN

In [49]:
# Remove duplicate columns, if any
pivot_data = pivot_data.loc[:, ~pivot_data.columns.duplicated()]
# Replace NaN values with 0s
pivot_data.fillna(0, inplace=True)

In [50]:
# Group 'filtered_filtered_data' by both 'timestamp' and 'activity_encoded' and aggregate 'duration'
activity_grouped = filtered_data.groupby(['timestamp'])['activity'].first()

# Merge 'duration_grouped' with 'pivot_data' based on timestamp index
pivot_data = pivot_data.merge(activity_grouped, how='left', on='timestamp')

# Print the updated pivoted data with 'duration' as a column specific to each activity
print(pivot_data)

                    timestamp duration    accel_x    accel_y    accel_z  \
0     2017-06-29 09:57:08.396    00:20   0.000000   0.000000   0.000000   
1     2017-06-29 09:57:08.406    00:20 -10.173126  -1.471237   1.924393   
2     2017-06-29 09:57:08.589    00:20   0.000000   0.000000   0.000000   
3     2017-06-29 09:57:08.591    00:20   0.000000   0.000000   0.000000   
4     2017-06-29 09:57:13.436    00:20   4.726227 -10.251968   0.491104   
...                       ...      ...        ...        ...        ...   
24898 2017-07-13 20:03:58.637    00:41  -2.270111  -8.976822  -3.034347   
24899 2017-07-13 20:05:00.880    00:41   0.000000   0.000000   0.000000   
24900 2017-07-13 20:05:00.933    00:41  -3.145065  -8.022736  -4.604950   
24901 2017-07-13 20:21:31.322    00:41   0.000000   0.000000   0.000000   
24902 2017-07-13 20:21:31.358    00:41  -0.675690   2.700424  10.965744   

       gravity_x  gravity_y  gravity_z    gyro_x    gyro_y    gyro_z  \
0       0.000000   0.000000

In [51]:
# Read the CSV file into a DataFrame
smartphone_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/smartphone_full.csv')

# Print the DataFrame
print(smartphone_full)

          index         source                timestamp  \
0       1316828  step_detector  2017-06-29 07:45:16.506   
1       1316863  step_detector  2017-06-29 07:45:27.858   
2       1316867  step_detector  2017-06-29 07:45:28.940   
3       1316870  step_detector  2017-06-29 07:45:31.018   
4       1316880  step_detector  2017-06-29 07:45:32.327   
...         ...            ...                      ...   
617959  1316814          audio  2017-07-13 20:13:33.671   
617960  1316817          audio  2017-07-13 20:13:34.663   
617961  1316819          audio  2017-07-13 20:13:35.663   
617962  1316823          audio  2017-07-13 20:13:36.668   
617963  1316824          audio  2017-07-13 20:13:37.668   

                                            values  
0                                          ['1.0']  
1                                          ['1.0']  
2                                          ['1.0']  
3                                          ['1.0']  
4                         

In [52]:
import pandas as pd

# Read the CSV file into a DataFrame
smartphone_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/smartphone_full.csv')

# Explode the lists in the 'values' column into separate rows
smartphone_full_expanded = smartphone_full.explode('values')

# Pivot the DataFrame
pivot_table_s = pd.pivot_table(smartphone_full_expanded,
                               index='timestamp',
                               columns='source',
                               values='values',
                               aggfunc='first')

# Rename the columns
pivot_table_s.rename(columns={'audio': 'audio_values', 'step_detector': 'step_detector_values'}, inplace=True)

# Reset index
pivot_table_s.reset_index(inplace=True)

# Define the pathway for the CSV file
pivot_table_s_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s.csv'

# Save the pivoted DataFrame to a CSV file
pivot_table_s.to_csv(pivot_table_s_pathway, index=False)

# Print the file pathway where the CSV file is saved
print("CSV file saved to:", pivot_table_s_pathway)

CSV file saved to: /Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s.csv


In [53]:
# Convert the 'audio_values' column from list to string and remove square brackets
pivot_table_s['audio_values'] = pivot_table_s['audio_values'].astype(str).str.strip('[]')

# Split the 'audio_values' column into separate columns
audio_values_split = pivot_table_s['audio_values'].str.split(',', expand=True)

# Rename the new columns
audio_values_split.columns = ['audio_value_1', 'audio_value_2', 'audio_value_3', 'audio_value_4']

# Concatenate the new columns with the original DataFrame
pivot_table_s_split = pd.concat([pivot_table_s.drop('audio_values', axis=1), audio_values_split], axis=1)

# Print the updated DataFrame
print(pivot_table_s_split)

                      timestamp step_detector_values audio_value_1  \
0       2017-06-29 07:45:16.506              ['1.0']           nan   
1       2017-06-29 07:45:17.407                  NaN         '0.0'   
2       2017-06-29 07:45:18.407                  NaN       '227.0'   
3       2017-06-29 07:45:19.407                  NaN       '590.0'   
4       2017-06-29 07:45:20.446                  NaN      '1724.0'   
...                         ...                  ...           ...   
617948  2017-07-13 20:13:33.671                  NaN      '1956.0'   
617949  2017-07-13 20:13:34.663                  NaN      '2673.0'   
617950  2017-07-13 20:13:35.663                  NaN      '2340.0'   
617951  2017-07-13 20:13:36.668                  NaN      '2046.0'   
617952  2017-07-13 20:13:37.668                  NaN      '2441.0'   

       audio_value_2 audio_value_3 audio_value_4  
0               None          None          None  
1              '0.0'         '0.0'     '32767.0'  
2     

In [22]:
pivot_table_s_split.fillna(0, inplace=True)

In [54]:
# Remove square brackets and single quotes from the values in 'step_detector_values' column
pivot_table_s_split['step_detector_values'] = pivot_table_s_split['step_detector_values'].str.strip("[]").str.replace("'", "")

# Convert the values in 'step_detector_values' column to float
pivot_table_s_split['step_detector_values'] = pivot_table_s_split['step_detector_values'].astype(float)

In [55]:
pivot_table_s_split['audio_value_1'] = pivot_table_s_split['audio_value_1'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_2'] = pivot_table_s_split['audio_value_2'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_3'] = pivot_table_s_split['audio_value_3'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_4'] = pivot_table_s_split['audio_value_4'].str.replace("'", '').astype(float)

In [56]:
# Convert the 'timestamp' column to datetime data type
pivot_table_s_split['timestamp'] = pd.to_datetime(pivot_table_s_split['timestamp'])

In [57]:
pivot_table_s_split.fillna(0, inplace=True)

In [58]:
# Convert timestamp columns to datetime objects
activity_type_full['from_1'] = pd.to_datetime(activity_type_full['from_1'])
pivot_table_s_split['timestamp'] = pd.to_datetime(pivot_table_s_split['timestamp'])

# Merge the data frames based on timestamps
merged_data_s = pd.merge_asof(pivot_table_s_split.sort_values('timestamp'), 
                             activity_type_full.sort_values('from_1'),
                             left_on='timestamp',
                             right_on='from_1',
                             direction='backward')

# Optionally, drop redundant columns or rename them as needed
merged_data_s.drop(columns=['from_1', 'to_1'], inplace=True)
merged_data_s.rename(columns={'activity_type': 'activity'}, inplace=True)

# Fill NaN values with 0s
merged_data_s.fillna(0, inplace=True)

# Print or further process the merged data
print(merged_data_s)

                     timestamp  step_detector_values  audio_value_1  \
0      2017-06-29 07:45:16.506                   1.0            0.0   
1      2017-06-29 07:45:17.407                   0.0            0.0   
2      2017-06-29 07:45:18.407                   0.0          227.0   
3      2017-06-29 07:45:19.407                   0.0          590.0   
4      2017-06-29 07:45:20.446                   0.0         1724.0   
...                        ...                   ...            ...   
617948 2017-07-13 20:13:33.671                   0.0         1956.0   
617949 2017-07-13 20:13:34.663                   0.0         2673.0   
617950 2017-07-13 20:13:35.663                   0.0         2340.0   
617951 2017-07-13 20:13:36.668                   0.0         2046.0   
617952 2017-07-13 20:13:37.668                   0.0         2441.0   

        audio_value_2  audio_value_3  audio_value_4     activity duration  
0             0.00000            0.0            0.0            0       

In [71]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
merged_data_s_f = merged_data_s[(merged_data_s['activity'] != 0) & (merged_data_s['duration'] != 0)]

In [62]:
pivot_data_2 = pivot_data.drop(columns = ['duration', 'activity'])

In [64]:
merged_data_s

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity,duration
0,2017-06-29 07:45:16.506,1.0,0.0,0.00000,0.0,0.0,0,0
1,2017-06-29 07:45:17.407,0.0,0.0,0.00000,0.0,32767.0,0,0
2,2017-06-29 07:45:18.407,0.0,227.0,113.50000,227.0,227.0,0,0
3,2017-06-29 07:45:19.407,0.0,590.0,272.33334,590.0,227.0,0,0
4,2017-06-29 07:45:20.446,0.0,1724.0,635.25000,1724.0,227.0,0,0
...,...,...,...,...,...,...,...,...
617948,2017-07-13 20:13:33.671,0.0,1956.0,5246.24850,32767.0,30.0,Video games,00:41
617949,2017-07-13 20:13:34.663,0.0,2673.0,5246.18160,32767.0,30.0,Video games,00:41
617950,2017-07-13 20:13:35.663,0.0,2340.0,5246.10640,32767.0,30.0,Video games,00:41
617951,2017-07-13 20:13:36.668,0.0,2046.0,5246.02400,32767.0,30.0,Video games,00:41


In [63]:
pivot_data_2

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z
0,2017-06-29 09:57:08.396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.351562,0.686813,0.476898,0.000000,0.000000,0.000000
1,2017-06-29 09:57:08.406,-10.173126,-1.471237,1.924393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2017-06-29 09:57:08.589,0.000000,0.000000,0.000000,4.585228,-8.101572,3.083927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2017-06-29 09:57:08.591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.280773,-0.710495,-0.692953
4,2017-06-29 09:57:13.436,4.726227,-10.251968,0.491104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24898,2017-07-13 20:03:58.637,-2.270111,-8.976822,-3.034347,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24899,2017-07-13 20:05:00.880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075684,-0.002960,0.047028,0.000000,0.000000,0.000000
24900,2017-07-13 20:05:00.933,-3.145065,-8.022736,-4.604950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24901,2017-07-13 20:21:31.322,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.387283,0.058716,0.566345,0.000000,0.000000,0.000000


In [67]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
merged_data_s_f = merged_data_s[(merged_data_s['activity'] != 0) & (merged_data_s['duration'] != 0)]

In [68]:
merged_data_s_f 

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity,duration
13030,2017-06-29 09:57:00.280,1.0,0.0,0.0000,0.0,0.0,Eat,00:20
13031,2017-06-29 09:57:00.448,0.0,8513.0,7546.2360,32767.0,45.0,Eat,00:20
13032,2017-06-29 09:57:01.000,1.0,0.0,0.0000,0.0,0.0,Eat,00:20
13033,2017-06-29 09:57:01.447,0.0,8273.0,7546.3276,32767.0,45.0,Eat,00:20
13034,2017-06-29 09:57:01.521,1.0,0.0,0.0000,0.0,0.0,Eat,00:20
...,...,...,...,...,...,...,...,...
617948,2017-07-13 20:13:33.671,0.0,1956.0,5246.2485,32767.0,30.0,Video games,00:41
617949,2017-07-13 20:13:34.663,0.0,2673.0,5246.1816,32767.0,30.0,Video games,00:41
617950,2017-07-13 20:13:35.663,0.0,2340.0,5246.1064,32767.0,30.0,Video games,00:41
617951,2017-07-13 20:13:36.668,0.0,2046.0,5246.0240,32767.0,30.0,Video games,00:41


In [69]:
# Convert the timestamp columns to datetime objects
pivot_data_2['timestamp'] = pd.to_datetime(pivot_data_2['timestamp'])
merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])

# Merge the dataframes based on timestamps
merged_data_s_f1 = pd.merge(pivot_data_2, merged_data_s_f, on='timestamp', how='outer')

# Sort the dataframe by timestamp
merged_data_s_f1.sort_values(by='timestamp', inplace=True)

# Reset the index
merged_data_s_f1.reset_index(drop=True, inplace=True)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/3578172377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])


In [70]:
merged_data_s_f1.fillna(0, inplace=True)

# Define the pathway for the CSV file
merged_data_s_f1_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/merged_data_s_f1.csv'

# Save the pivoted DataFrame to a CSV file
merged_data_s_f1.to_csv(merged_data_s_f1_pathway, index=False)

In [71]:
merged_data_s_f1['duration_numeric'] = merged_data_s_f1['duration'].astype(str).str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]) if len(x) == 2 else np.nan)

In [72]:
merged_data_s_f1.drop(columns = ['duration'])

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity,duration_numeric
0,2017-06-29 09:57:00.280,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0000,0.0,0.0,Eat,20.0
1,2017-06-29 09:57:00.448,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,8513.0,7546.2360,32767.0,45.0,Eat,20.0
2,2017-06-29 09:57:01.000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0000,0.0,0.0,Eat,20.0
3,2017-06-29 09:57:01.447,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,8273.0,7546.3276,32767.0,45.0,Eat,20.0
4,2017-06-29 09:57:01.521,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0000,0.0,0.0,Eat,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629787,2017-07-13 20:13:35.663,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2340.0,5246.1064,32767.0,30.0,Video games,41.0
629788,2017-07-13 20:13:36.668,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2046.0,5246.0240,32767.0,30.0,Video games,41.0
629789,2017-07-13 20:13:37.668,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2441.0,5245.9517,32767.0,30.0,Video games,41.0
629790,2017-07-13 20:21:31.322,0.00000,0.000000,0.000000,0.0,0.0,0.0,-0.387283,0.058716,0.566345,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0,NaN


In [73]:
# Filter out timestamps where at least one of the accelerometer or gyro values is not 0
condition_1 = (merged_data_s_f1[['accel_x', 'accel_y', 'accel_z', 'gravity_x', 'gravity_y', 'gravity_z', 
                                 'gyro_x', 'gyro_y', 'gyro_z', 'linear_accel_x', 'linear_accel_y', 'linear_accel_z']] != 0).any(axis=1)

# Filter out timestamps where at least one of the step detector or audio values is not 0
condition_2 = (merged_data_s_f1[['step_detector_values', 'audio_value_1', 'audio_value_2', 
                                 'audio_value_3', 'audio_value_4']] != 0).any(axis=1)

# Apply both conditions
filtered_merged_data_s_f1= merged_data_s_f1[condition_1 & condition_2]

# Reset the index
filtered_merged_data_s_f1.reset_index(drop=True, inplace=True)

In [74]:
# Convert duration to numeric (minutes)
filtered_merged_data_s_f1['duration_numeric'] = filtered_merged_data_s_f1['duration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/3655484280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_merged_data_s_f1['duration_numeric'] = filtered_merged_data_s_f1['duration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))


In [75]:
filtered_merged_data_s_f1 = filtered_merged_data_s_f1.drop(columns = ['duration'])

In [76]:
filtered_merged_data_s_f1 

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity,duration_numeric
0,2017-06-29 10:10:19.102,-10.483658,0.005508,-0.231705,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0000,0.0,0.0,Eat,20
1,2017-06-29 18:19:03.450,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.050703,0.420142,-0.185971,0.0,7093.0,7384.0713,32767.0,33.0,Walking&party,277
2,2017-06-29 18:55:10.448,0.000000,0.000000,0.000000,-6.544652,-5.040787,-5.284731,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,20530.0,7567.0747,32767.0,33.0,Walking&party,277
3,2017-06-30 10:10:47.629,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.269095,-1.888973,4.452594,1.0,0.0,0.0000,0.0,0.0,Running,21
4,2017-06-30 11:51:53.676,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.163757,0.020248,-1.354340,0.000000,0.000000,0.000000,1.0,0.0,0.0000,0.0,0.0,Train,22
5,2017-06-30 13:20:59.148,0.000000,0.000000,0.000000,-9.713084,-0.624524,1.198473,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,32767.0,4367.6350,32767.0,46.0,In computer,86
6,2017-07-01 14:44:04.928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001785,0.000366,0.000427,0.000000,0.000000,0.000000,0.0,9542.0,3135.6013,32767.0,42.0,Sleep,271
7,2017-07-01 20:40:08.928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427185,0.120743,0.038269,0.000000,0.000000,0.000000,0.0,1488.0,2043.0955,32767.0,31.0,In computer,20
8,2017-07-03 18:21:59.275,8.757355,1.651520,4.694489,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,472.0,1857.6748,32767.0,42.0,Phone was out of the pocket (forgot),68
9,2017-07-04 14:12:14.417,-11.908920,-2.038315,1.776794,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,17940.0,6569.9155,32767.0,94.0,Walk,3


In [77]:
# Filter out timestamps where activity and duration = 0
condition11 = (merged_data_s_f1['activity'] == 0)

# Apply both conditions
filtered_merged_data_s_f2 = merged_data_s_f1[condition11]

# Reset the index
filtered_merged_data_s_f2.reset_index(drop=True, inplace=True)

In [78]:
# Reset the index
filtered_merged_data_s_f2.fillna(0, inplace=True)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/1448412941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_merged_data_s_f2.fillna(0, inplace=True)


In [79]:
filtered_merged_data_s_f2 = filtered_merged_data_s_f2.drop(columns=['duration', 'activity'])
filtered_merged_data_s_f2

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,duration_numeric
0,2017-06-29 09:57:08.396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.351562,0.686813,0.476898,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-06-29 09:57:08.406,-10.173126,-1.471237,1.924393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-06-29 09:57:08.589,0.000000,0.000000,0.000000,4.585228,-8.101572,3.083927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-06-29 09:57:08.591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.280773,-0.710495,-0.692953,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-06-29 09:57:13.436,4.726227,-10.251968,0.491104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24864,2017-07-13 20:03:58.637,-2.270111,-8.976822,-3.034347,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
24865,2017-07-13 20:05:00.880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075684,-0.002960,0.047028,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
24866,2017-07-13 20:05:00.933,-3.145065,-8.022736,-4.604950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
24867,2017-07-13 20:21:31.322,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.387283,0.058716,0.566345,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
filtered_merged_data_s_f1['activity_encoded'] = label_encoder.fit_transform(filtered_merged_data_s_f1['activity'])

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report

# Assuming X contains the features and y contains the target variable

# Split the data into features (X) and target variable (y)
X = filtered_merged_data_s_f1[['accel_x', 'accel_z', 'gravity_x', 'gravity_y', 'gyro_x', 'gyro_y', 'gyro_z', 'audio_value_2', 'audio_value_4', 'duration_numeric']]
y = filtered_merged_data_s_f1['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(y.unique()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' for integer-encoded targets
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=150, validation_split=0.2)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activity_all.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

2024-03-16 17:58:09.361843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/150
1/1 [==============================] - 3s 3s/step - loss: 2.9805 - accuracy: 0.0952 - val_loss: 2.5827 - val_accuracy: 0.0000e+00
Epoch 2/150
1/1 [==============================] - 0s 121ms/step - loss: 2.8701 - accuracy: 0.1429 - val_loss: 2.5791 - val_accuracy: 0.0000e+00
Epoch 3/150
1/1 [==============================] - 0s 110ms/step - loss: 3.2773 - accuracy: 0.1429 - val_loss: 2.5758 - val_accuracy: 0.0000e+00
Epoch 4/150
1/1 [==============================] - 0s 113ms/step - loss: 3.2450 - accuracy: 0.0000e+00 - val_loss: 2.5720 - val_accuracy: 0.0000e+00
Epoch 5/150
1/1 [==============================] - 0s 75ms/step - loss: 3.0265 - accuracy: 0.0000e+00 - val_loss: 2.5693 - val_accuracy: 0.0000e+00
Epoch 6/150
1/1 [==============================] - 0s 150ms/step - loss: 2.6848 - accuracy: 0.0952 - val_loss: 2.5643 - val_accuracy: 0.0000e+00
Epoch 7/150
1/1 [==============================] - 0s 75ms/step - loss: 2.9924 - accuracy: 0.0952 - val_loss: 2.5600 - val_acc

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

In [106]:
from tensorflow.keras.models import load_model, save_model
from sklearn.preprocessing import LabelEncoder

# Load the previously saved model
model = load_model('/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activity_all.keras')

# Prepare the features from the unlabeled data
X_unlabeled = filtered_merged_data_s_f2[['accel_x', 'accel_z', 'gravity_x', 'gravity_y', 'gyro_x', 'gyro_y', 'gyro_z', 'audio_value_2', 'audio_value_4', 'duration_numeric']]

# Preprocess unlabeled data (assuming you've already standardized the features)
X_unlabeled_scaled = scaler.transform(X_unlabeled)

# Make predictions
y_pred = model.predict(X_unlabeled_scaled)

# Inverse transform the predicted labels
decoded_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))

# Add the decoded labels to the DataFrame
filtered_merged_data_s_f2['predicted_activity'] = decoded_labels

778/778 [==============================] - 2s 2ms/step


In [81]:
filtered_merged_data_s_f2

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,gyro_x,gyro_y,gyro_z,linear_accel_x,linear_accel_y,linear_accel_z,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,duration_numeric
0,2017-06-29 09:57:08.396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.351562,0.686813,0.476898,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-06-29 09:57:08.406,-10.173126,-1.471237,1.924393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-06-29 09:57:08.589,0.000000,0.000000,0.000000,4.585228,-8.101572,3.083927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-06-29 09:57:08.591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.280773,-0.710495,-0.692953,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-06-29 09:57:13.436,4.726227,-10.251968,0.491104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24864,2017-07-13 20:03:58.637,-2.270111,-8.976822,-3.034347,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
24865,2017-07-13 20:05:00.880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075684,-0.002960,0.047028,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
24866,2017-07-13 20:05:00.933,-3.145065,-8.022736,-4.604950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
24867,2017-07-13 20:21:31.322,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.387283,0.058716,0.566345,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


I will now continue with the smartglasses data. 

In [82]:
glasses

,NUM,DATE,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z,EOG_L,EOG_R,EOG_H,EOG_V
0,1,2017/06/29 08:00:00.406,-3242,10396,-6489,9117,2018,-819,-330,-186,-144,258
1,3,2017/06/29 08:00:00.426,-4014,11348,-15131,3359,-882,-2009,-305,-156,-149,230
2,5,2017/06/29 08:00:00.446,-3223,10918,-8496,-460,211,-2631,-64,84,-148,-10
3,7,2017/06/29 08:00:00.466,-5255,10690,-10854,1152,333,-3799,314,432,-118,-373
4,9,2017/06/29 08:00:00.486,-560,14164,-8619,-327,488,-2833,577,643,-66,-610
...,...,...,...,...,...,...,...,...,...,...,...,...
3975261,285647,2017/07/13 22:20:30.894,-5345,9790,-12500,336,638,-365,-186,-32,-154,109
3975262,285649,2017/07/13 22:20:30.914,-5280,9682,-12471,581,812,-271,-183,-28,-155,105
3975263,285651,2017/07/13 22:20:30.934,-5170,9579,-12492,828,963,-214,-176,-19,-157,97
3975264,285652,2017/07/13 22:20:30.944,-5082,9531,-12473,881,1062,-189,-159,-4,-155,81


In [83]:
glasses = glasses.drop(columns= ['NUM', 'EOG_L', 'EOG_R', 'EOG_H', 'EOG_V', ])

In [84]:
glasses

,DATE,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z
0,2017/06/29 08:00:00.406,-3242,10396,-6489,9117,2018,-819
1,2017/06/29 08:00:00.426,-4014,11348,-15131,3359,-882,-2009
2,2017/06/29 08:00:00.446,-3223,10918,-8496,-460,211,-2631
3,2017/06/29 08:00:00.466,-5255,10690,-10854,1152,333,-3799
4,2017/06/29 08:00:00.486,-560,14164,-8619,-327,488,-2833
...,...,...,...,...,...,...,...
3975261,2017/07/13 22:20:30.894,-5345,9790,-12500,336,638,-365
3975262,2017/07/13 22:20:30.914,-5280,9682,-12471,581,812,-271
3975263,2017/07/13 22:20:30.934,-5170,9579,-12492,828,963,-214
3975264,2017/07/13 22:20:30.944,-5082,9531,-12473,881,1062,-189


In [85]:
# Convert timestamp columns to datetime objects
activity_type_full['from_1'] = pd.to_datetime(activity_type_full['from_1'])
glasses['DATE'] = pd.to_datetime(glasses['DATE'])

# Merge the data frames based on timestamps
merged_data_glasses = pd.merge_asof(glasses.sort_values('DATE'), 
                             activity_type_full.sort_values('from_1'),
                             left_on='DATE',
                             right_on='from_1',
                             direction='backward')

# Optionally, drop redundant columns or rename them as needed
merged_data_glasses.drop(columns=['from_1', 'to_1'], inplace=True)
merged_data_glasses.rename(columns={'activity_type': 'activity'}, inplace=True)

# Fill NaN values with 0s
merged_data_glasses.fillna(0, inplace=True)

# Print or further process the merged data
print(merged_data_glasses)

                           DATE  ACC_X  ACC_Y  ACC_Z  GYRO_X  GYRO_Y  GYRO_Z  \
0       2017-06-29 08:00:00.406  -3242  10396  -6489    9117    2018    -819   
1       2017-06-29 08:00:00.426  -4014  11348 -15131    3359    -882   -2009   
2       2017-06-29 08:00:00.446  -3223  10918  -8496    -460     211   -2631   
3       2017-06-29 08:00:00.466  -5255  10690 -10854    1152     333   -3799   
4       2017-06-29 08:00:00.486   -560  14164  -8619    -327     488   -2833   
...                         ...    ...    ...    ...     ...     ...     ...   
3975261 2017-07-13 22:20:30.894  -5345   9790 -12500     336     638    -365   
3975262 2017-07-13 22:20:30.914  -5280   9682 -12471     581     812    -271   
3975263 2017-07-13 22:20:30.934  -5170   9579 -12492     828     963    -214   
3975264 2017-07-13 22:20:30.944  -5082   9531 -12473     881    1062    -189   
3975265 2017-07-13 22:20:30.964  -4968   9414 -12622    1052    1176    -154   

            activity duration  
0      

In [86]:
merged_data_glasses

,DATE,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z,activity,duration
0,2017-06-29 08:00:00.406,-3242,10396,-6489,9117,2018,-819,0,0
1,2017-06-29 08:00:00.426,-4014,11348,-15131,3359,-882,-2009,0,0
2,2017-06-29 08:00:00.446,-3223,10918,-8496,-460,211,-2631,0,0
3,2017-06-29 08:00:00.466,-5255,10690,-10854,1152,333,-3799,0,0
4,2017-06-29 08:00:00.486,-560,14164,-8619,-327,488,-2833,0,0
...,...,...,...,...,...,...,...,...,...
3975261,2017-07-13 22:20:30.894,-5345,9790,-12500,336,638,-365,Video games,00:41
3975262,2017-07-13 22:20:30.914,-5280,9682,-12471,581,812,-271,Video games,00:41
3975263,2017-07-13 22:20:30.934,-5170,9579,-12492,828,963,-214,Video games,00:41
3975264,2017-07-13 22:20:30.944,-5082,9531,-12473,881,1062,-189,Video games,00:41


In [87]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_data_glasses = merged_data_glasses[(merged_data_glasses['activity'] != 0) & (merged_data_glasses['duration'] != 0)]

In [88]:
filtered_data_glasses 

,DATE,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z,activity,duration
60733,2017-06-29 19:59:15.360,-3909,10146,-12051,-498,237,-40,Walking&party,04:37
60734,2017-06-29 19:59:15.380,-3891,10106,-12094,-501,305,-85,Walking&party,04:37
60735,2017-06-29 19:59:15.400,-3861,10110,-12147,-475,320,-83,Walking&party,04:37
60736,2017-06-29 19:59:15.420,-3820,10161,-12109,-425,346,-47,Walking&party,04:37
60737,2017-06-29 19:59:15.440,-3772,10088,-12118,-361,365,18,Walking&party,04:37
...,...,...,...,...,...,...,...,...,...
3975261,2017-07-13 22:20:30.894,-5345,9790,-12500,336,638,-365,Video games,00:41
3975262,2017-07-13 22:20:30.914,-5280,9682,-12471,581,812,-271,Video games,00:41
3975263,2017-07-13 22:20:30.934,-5170,9579,-12492,828,963,-214,Video games,00:41
3975264,2017-07-13 22:20:30.944,-5082,9531,-12473,881,1062,-189,Video games,00:41


In [91]:
# Convert duration to numeric (minutes)
filtered_data_glasses['duration'] = filtered_data_glasses['duration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

AttributeError: Can only use .str accessor with string values!

In [92]:
filtered_data_glasses.columns

Index(['DATE', 'ACC_X', 'ACC_Y', 'ACC_Z', 'GYRO_X', 'GYRO_Y', 'GYRO_Z',
       'activity', 'duration'],
      dtype='object')

In [93]:
filtered_data_glasses['accel_x'] = filtered_data_glasses['ACC_X']
filtered_data_glasses['accel_y'] = filtered_data_glasses['ACC_Y']
filtered_data_glasses['accel_z'] = filtered_data_glasses['ACC_Z']
filtered_data_glasses['gyro_x'] = filtered_data_glasses['GYRO_X']
filtered_data_glasses['gyro_y'] = filtered_data_glasses['GYRO_Y']
filtered_data_glasses['gyro_z'] = filtered_data_glasses['GYRO_Z']
filtered_data_glasses['duration_numeric'] = filtered_data_glasses['duration']

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/894586061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_glasses['accel_x'] = filtered_data_glasses['ACC_X']
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/894586061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_glasses['accel_y'] = filtered_data_glasses['ACC_Y']
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/894586061.py:3: SettingWithCopyWarning: 
A value is trying to be

In [ ]:
filtered_data_glasses = filtered_data_glasses.drop(columns = ['ACC_Z', 'ACC_X', 'ACC_Y', 'GYRO_X', 'GYRO_Y', 'GYRO_Z'])

In [94]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
filtered_data_glasses['activity_encoded'] = label_encoder.fit_transform(filtered_data_glasses['activity'])

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_1536/256512229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_glasses['activity_encoded'] = label_encoder.fit_transform(filtered_data_glasses['activity'])


In [96]:
filtered_data_glasses.drop(columns =['duration', 'ACC_X', 'ACC_Y', 'ACC_Z', 'GYRO_X', 'GYRO_Y', 'GYRO_Z'])

,DATE,activity,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,duration_numeric,activity_encoded
60733,2017-06-29 19:59:15.360,Walking&party,-3909,10146,-12051,-498,237,-40,277,13
60734,2017-06-29 19:59:15.380,Walking&party,-3891,10106,-12094,-501,305,-85,277,13
60735,2017-06-29 19:59:15.400,Walking&party,-3861,10110,-12147,-475,320,-83,277,13
60736,2017-06-29 19:59:15.420,Walking&party,-3820,10161,-12109,-425,346,-47,277,13
60737,2017-06-29 19:59:15.440,Walking&party,-3772,10088,-12118,-361,365,18,277,13
...,...,...,...,...,...,...,...,...,...,...
3975261,2017-07-13 22:20:30.894,Video games,-5345,9790,-12500,336,638,-365,41,11
3975262,2017-07-13 22:20:30.914,Video games,-5280,9682,-12471,581,812,-271,41,11
3975263,2017-07-13 22:20:30.934,Video games,-5170,9579,-12492,828,963,-214,41,11
3975264,2017-07-13 22:20:30.944,Video games,-5082,9531,-12473,881,1062,-189,41,11


In [97]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report

# Split the data into features (X) and target variable (y)
X = filtered_data_glasses[['accel_x', 'accel_z', 'accel_y', 'gyro_x', 'gyro_y', 'gyro_z', 'duration_numeric']]
y = filtered_data_glasses['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(y.unique()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' for integer-encoded targets
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activity_all.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

2024-03-17 08:29:37.883651: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
78291/78291 [==============================] - 316s 4ms/step - loss: 1.1645 - accuracy: 0.5683 - val_loss: 0.8622 - val_accuracy: 0.6977
Epoch 2/20
78291/78291 [==============================] - 346s 4ms/step - loss: 1.0764 - accuracy: 0.5979 - val_loss: 0.8342 - val_accuracy: 0.7080
Epoch 3/20
78291/78291 [==============================] - 538s 7ms/step - loss: 1.0620 - accuracy: 0.6033 - val_loss: 0.8341 - val_accuracy: 0.7024
Epoch 4/20
78291/78291 [==============================] - 293s 4ms/step - loss: 1.0544 - accuracy: 0.6076 - val_loss: 0.8468 - val_accuracy: 0.7014
Epoch 5/20
78291/78291 [==============================] - 293s 4ms/step - loss: 1.0459 - accuracy: 0.6116 - val_loss: 0.8269 - val_accuracy: 0.7016
Epoch 6/20
78291/78291 [==============================] - 832s 11ms/step - loss: 1.0391 - accuracy: 0.6133 - val_loss: 0.8103 - val_accuracy: 0.7127
Epoch 7/20
78291/78291 [==============================] - 304s 4ms/step - loss: 1.0376 - accuracy: 0.6144 - val

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.90      0.95     78264
           1       0.66      0.76      0.71     40377
           2       0.62      0.83      0.71    148944
           3       0.56      0.43      0.49     67179
           4       1.00      0.41      0.58     27989
           5       0.82      0.78      0.80     37723
           6       0.96      0.30      0.45     24908
           7       0.00      0.00      0.00      1713
           8       0.00      0.00      0.00     10409
           9       0.38      0.67      0.48     52978
          10       0.00      0.00      0.00     34673
          11       0.90      1.00      0.95    154981
          12       0.75      0.50      0.60     51318
          13       0.81      1.00      0.89     50734
          14       0.00      0.00      0.00       717

    accuracy                           0.73    782907
   macro avg       0.56      0.50      0.51    782907
weighted avg       0.71   

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


As we can see the more data we have the better the model learns. 

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight


# Split the data into features (X) and target variable (y)
X = filtered_data_glasses[['accel_x', 'accel_z', 'accel_y', 'gyro_x', 'gyro_y', 'gyro_z', 'duration_numeric']]
y = filtered_data_glasses['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(y_train)), activation='softmax') 
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=3, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/3
78291/78291 [==============================] - 329s 4ms/step - loss: 1.5014 - accuracy: 0.3617 - val_loss: 1.1848 - val_accuracy: 0.5250
Epoch 2/3
78291/78291 [==============================] - 349s 4ms/step - loss: 1.2966 - accuracy: 0.4148 - val_loss: 1.2464 - val_accuracy: 0.5321
Epoch 3/3
24466/24466 [==============================] - 55s 2ms/step - loss: 1.1996 - accuracy: 0.5104
Test Accuracy: 0.5104169249534607
24466/24466 [==============================] - 49s 2ms/step
              precision    recall  f1-score   support

           0       1.00      0.15      0.26     78264
           1       0.50      1.00      0.67     40377
           2       0.89      0.27      0.41    148944
           3       0.50      0.63      0.56     67179
           4       0.47      0.73      0.57     27989
           5       0.47      0.81      0.59     37723
           6       0.89      0.98      0.93     24908
           7       0.02      1.00      0.04      1713
           8       0.

The result shows that we should ins some way use 'balanced' class weighting because even minority classes achieve some accuracy as a result. 

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report

# Split the data into features (X) and target variable (y)
X = filtered_data_glasses[['accel_x', 'accel_z', 'accel_y', 'gyro_x', 'gyro_y', 'gyro_z', 'duration_numeric']]
y = filtered_data_glasses['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


# Load the previously trained model
model_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activity_all.keras"
model = load_model(model_path)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=3, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict)

# Save the fine-tuned model
model_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activity_all.keras"
save_model(model, model_path)

# Evaluate the fine-tuned model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy after fine-tuning:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)

Epoch 1/3
78291/78291 [==============================] - 344s 4ms/step - loss: 1.2446 - accuracy: 0.4646 - val_loss: 1.1380 - val_accuracy: 0.4503
Epoch 2/3
78291/78291 [==============================] - 308s 4ms/step - loss: 1.2245 - accuracy: 0.4608 - val_loss: 1.0769 - val_accuracy: 0.4764
Epoch 3/3
24466/24466 [==============================] - 62s 3ms/step - loss: 1.0244 - accuracy: 0.6340
Test Accuracy after fine-tuning: 0.6339603662490845
24466/24466 [==============================] - 59s 2ms/step


In [105]:
print(report)

              precision    recall  f1-score   support

           0       1.00      0.15      0.26     78264
           1       0.50      1.00      0.66     40377
           2       0.90      0.42      0.58    148944
           3       0.49      0.68      0.57     67179
           4       0.60      0.73      0.66     27989
           5       0.67      0.83      0.74     37723
           6       0.89      0.99      0.93     24908
           7       0.04      0.99      0.09      1713
           8       0.21      0.98      0.34     10409
           9       0.49      0.72      0.58     52978
          10       0.74      0.36      0.48     34673
          11       0.98      0.75      0.85    154981
          12       0.70      0.57      0.63     51318
          13       0.98      0.98      0.98     50734
          14       0.03      0.91      0.05       717

    accuracy                           0.63    782907
   macro avg       0.61      0.74      0.56    782907
weighted avg       0.79   